In [31]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs
import tqdm

import pyet
import spei

In [32]:
sys.path.append(r'C:\Users\Pooya\w\GitHub\ShiraziPooya\DroughtMonitoringIran')

GEO_DATA_PATH = "../assets/geo_data"
DATABASE_PATH = "../database/database.db"

In [33]:
conn = sqlite3.connect(DATABASE_PATH)

precip_monthly = pd.read_sql(sql='SELECT * FROM gee_precip_monthly', con=conn)
precip_monthly['date'] = pd.to_datetime(precip_monthly['date'])
precip_monthly.drop(columns=['lat', 'lon', 'station_elevation'], inplace=True)

pet_monthly = pd.read_sql(sql='SELECT * FROM gee_pet_monthly', con=conn)
pet_monthly['date'] = pd.to_datetime(pet_monthly['date'])
pet_monthly.drop(columns=['lat', 'lon', 'station_elevation'], inplace=True)

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [34]:
print(precip_monthly.columns)
precip_monthly

Index(['region_id', 'region_name', 'station_id', 'station_name', 'date',
       'CHIRPS', 'ERA5', 'GPM', 'PERSIANN_CDR', 'TRMM', 'TerraClimate'],
      dtype='object')


,region_id,region_name,station_id,station_name,date,CHIRPS,ERA5,GPM,PERSIANN_CDR,TRMM,TerraClimate
0,MASA,Mazandaran,40732,Ramsar,1980-01-31,NaN,282.246388,NaN,NaN,NaN,103.0
1,MASA,Mazandaran,40732,Ramsar,1980-02-29,NaN,162.290688,NaN,NaN,NaN,86.0
2,MASA,Mazandaran,40732,Ramsar,1980-03-31,NaN,151.580624,NaN,NaN,NaN,89.0
3,MASA,Mazandaran,40732,Ramsar,1980-04-30,NaN,84.181468,NaN,NaN,NaN,33.0
4,MASA,Mazandaran,40732,Ramsar,1980-05-31,NaN,84.903626,NaN,NaN,NaN,29.0
...,...,...,...,...,...,...,...,...,...,...,...
8245,MASA,Mazandaran,99361,Alasht,2025-06-30,19.601683,84.377884,7.920001,NaN,NaN,NaN
8246,MASA,Mazandaran,99361,Alasht,2025-07-31,19.137444,21.478749,13.392001,NaN,NaN,NaN
8247,MASA,Mazandaran,99361,Alasht,2025-08-31,36.370131,33.082195,NaN,NaN,NaN,NaN
8248,MASA,Mazandaran,99361,Alasht,2025-09-30,34.380717,150.438410,NaN,NaN,NaN,NaN


In [35]:
print(pet_monthly.columns)
pet_monthly

Index(['region_id', 'region_name', 'station_id', 'station_name', 'date',
       'MOD16A2GF'],
      dtype='object')


,region_id,region_name,station_id,station_name,date,MOD16A2GF
0,MASA,Mazandaran,40732,Ramsar,1999-12-31,NaN
1,MASA,Mazandaran,40732,Ramsar,2000-01-31,61.3875
2,MASA,Mazandaran,40732,Ramsar,2000-02-29,84.0125
3,MASA,Mazandaran,40732,Ramsar,2000-03-31,140.3375
4,MASA,Mazandaran,40732,Ramsar,2000-04-30,196.2250
...,...,...,...,...,...,...
4510,MASA,Mazandaran,99361,Alasht,2024-08-31,225.5875
4511,MASA,Mazandaran,99361,Alasht,2024-09-30,153.4125
4512,MASA,Mazandaran,99361,Alasht,2024-10-31,124.9500
4513,MASA,Mazandaran,99361,Alasht,2024-11-30,73.1000


In [36]:
print(geoinfo.columns)
geoinfo

Index(['region_id', 'region_name', 'station_name', 'station_id', 'lat', 'lon',
       'station_elevation'],
      dtype='object')


,region_id,region_name,station_name,station_id,lat,lon,station_elevation
0,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0
1,MASA,Mazandaran,Amol,99309,36.48,52.47,23.7
2,MASA,Mazandaran,Babolsar,40736,36.70,52.64,-21.0
3,MASA,Mazandaran,Baladeh,99357,36.20,51.80,2120.0
4,MASA,Mazandaran,Bandar-E-Amirabad,99306,36.86,53.39,-20.0
5,MASA,Mazandaran,Galugah,99299,36.74,53.84,-10.0
6,MASA,Mazandaran,Gharakhil,40737,36.49,52.11,14.7
7,MASA,Mazandaran,Kiyasar,40760,36.25,53.55,1294.3
8,MASA,Mazandaran,Kojur,99348,36.39,51.73,1550.0
9,MASA,Mazandaran,Nowshahr,40734,36.66,51.47,-20.9


In [37]:
precip_models = ['CHIRPS', 'ERA5', 'GPM', 'PERSIANN_CDR', 'TRMM', 'TerraClimate']
pet_model = 'MOD16A2GF'

data_si = precip_monthly\
    .merge(pet_monthly, on=['region_id', 'region_name', 'station_name', 'station_id', 'date'], how='outer')\
    .set_index(['date'])
data_si
for pm in precip_models:
    data_si[f'PE_{pm}'] = data_si[f'{pm}'] - data_si[f'{pet_model}']

timescale = [1, 3, 6, 9, 12, 15, 18, 21, 24]
i = 1

for ts in tqdm.tqdm(timescale):
    for pm in tqdm.tqdm(precip_models):
         
        df_spi = data_si\
            .groupby(by=['region_id', 'region_name', 'station_id', 'station_name'])\
            .apply(
                lambda x: spei.spi(
                    series=x[f'{pm}'],
                    dist=scs.gamma,
                    prob_zero=True,
                    timescale=ts
                ),
                include_groups=False
            )\
            .reset_index()
        df_spi = df_spi.melt(id_vars=['region_id', 'region_name', 'station_id', 'station_name'], value_name=f'{pm}_SPI_{ts}')
        if i == 1:
            results = df_spi.copy()
        else:
            results = results.merge(df_spi, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')
        
        df_spei = data_si\
            .groupby(by=['region_id', 'region_name', 'station_id', 'station_name'])\
            .apply(
                lambda x: spei.spei(
                    series=x[f'PE_{pm}'],
                    dist=scs.fisk,
                    prob_zero=True,
                    timescale=ts
                ),
                include_groups=False
            )\
            .reset_index()
        df_spei = df_spei.melt(id_vars=['region_id', 'region_name', 'station_id', 'station_name'], value_name=f'{pm}_SPEI_{ts}')
        results = results.merge(df_spei, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')

        i += 1


results

100%|██████████| 9/9 [1:15:03<00:00, 500.39s/it]


,region_id,region_name,station_id,station_name,date,CHIRPS_SPI_1,CHIRPS_SPEI_1,ERA5_SPI_1,ERA5_SPEI_1,GPM_SPI_1,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,PERSIANN_CDR_SPI_24,PERSIANN_CDR_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TerraClimate_SPI_24,TerraClimate_SPEI_24
0,MASA,Mazandaran,40732,Ramsar,1980-01-31 00:00:00,NaN,NaN,1.716854,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MASA,Mazandaran,40732,Ramsar,1980-02-29 00:00:00,NaN,NaN,0.265937,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MASA,Mazandaran,40732,Ramsar,1980-03-31 00:00:00,NaN,NaN,0.660720,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASA,Mazandaran,40732,Ramsar,1980-04-30 00:00:00,NaN,NaN,0.027093,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MASA,Mazandaran,40732,Ramsar,1980-05-31 00:00:00,NaN,NaN,0.039086,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,MASA,Mazandaran,99361,Alasht,2025-06-30 00:00:00,-0.653235,NaN,-0.107414,NaN,-0.765715,...,-1.352864,NaN,-1.689297,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8246,MASA,Mazandaran,99361,Alasht,2025-07-31 00:00:00,-1.726621,NaN,-1.922230,NaN,-0.277952,...,-0.880545,NaN,-1.555055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8247,MASA,Mazandaran,99361,Alasht,2025-08-31 00:00:00,-0.616762,NaN,-1.102605,NaN,NaN,...,-1.583534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8248,MASA,Mazandaran,99361,Alasht,2025-09-30 00:00:00,-0.502448,NaN,0.457647,NaN,NaN,...,-1.176250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
results['date'] = pd.to_datetime(results['date'])
results

,region_id,region_name,station_id,station_name,date,CHIRPS_SPI_1,CHIRPS_SPEI_1,ERA5_SPI_1,ERA5_SPEI_1,GPM_SPI_1,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,PERSIANN_CDR_SPI_24,PERSIANN_CDR_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TerraClimate_SPI_24,TerraClimate_SPEI_24
0,MASA,Mazandaran,40732,Ramsar,1980-01-31,NaN,NaN,1.716854,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MASA,Mazandaran,40732,Ramsar,1980-02-29,NaN,NaN,0.265937,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MASA,Mazandaran,40732,Ramsar,1980-03-31,NaN,NaN,0.660720,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASA,Mazandaran,40732,Ramsar,1980-04-30,NaN,NaN,0.027093,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MASA,Mazandaran,40732,Ramsar,1980-05-31,NaN,NaN,0.039086,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,MASA,Mazandaran,99361,Alasht,2025-06-30,-0.653235,NaN,-0.107414,NaN,-0.765715,...,-1.352864,NaN,-1.689297,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8246,MASA,Mazandaran,99361,Alasht,2025-07-31,-1.726621,NaN,-1.922230,NaN,-0.277952,...,-0.880545,NaN,-1.555055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8247,MASA,Mazandaran,99361,Alasht,2025-08-31,-0.616762,NaN,-1.102605,NaN,NaN,...,-1.583534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8248,MASA,Mazandaran,99361,Alasht,2025-09-30,-0.502448,NaN,0.457647,NaN,NaN,...,-1.176250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
conn = sqlite3.connect(DATABASE_PATH)

results.to_sql('gee_di_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()